In [2]:
from astropy.table import Table, Column, join
from astropy.coordinates import SkyCoord
from astropy.table import Column
from astropy.io import ascii
from astropy import constants as const
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import Planck18 as cosmo
import pandas as pd
import pprint
import sys
import operator
from scipy import optimize
from scipy.optimize import curve_fit
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [4]:
input_files = ['allfields/aegis/aegis_3dhst.v4.1_f125w.galfit', 'allfields/aegis/aegis_3dhst.v4.1_f160w.galfit', 'allfields/cosmos/cosmos_3dhst.v4.1_f125w.galfit', 'allfields/cosmos/cosmos_3dhst.v4.1_f160w.galfit', 'allfields/goodsn/goodsn_3dhst.v4.1_f125w.galfit', 'allfields/goodsn/goodsn_3dhst.v4.1_f160w.galfit', 'allfields/goodss/goodss_3dhst.v4.1_f125w.galfit', 'allfields/goodss/goodss_3dhst.v4.1_f160w.galfit', 'allfields/uds/uds_3dhst.v4.1_f125w.galfit', 'allfields/uds/uds_3dhst.v4.1_f160w.galfit']
output_files = ['allfields/aegis/aegis_3dhst.v4.1_f125w_galfit.csv', 'allfields/aegis/aegis_3dhst.v4.1_f160w_galfit.csv', 'allfields/cosmos/cosmos_3dhst.v4.1_f125w_galfit.csv', 'allfields/cosmos/cosmos_3dhst.v4.1_f160w_galfit.csv', 'allfields/goodsn/goodsn_3dhst.v4.1_f125w_galfit.csv', 'allfields/goodsn/goodsn_3dhst.v4.1_f160w_galfit.csv', 'allfields/goodss/goodss_3dhst.v4.1_f125w_galfit.csv', 'allfields/goodss/goodss_3dhst.v4.1_f160w_galfit.csv', 'allfields/uds/uds_3dhst.v4.1_f125w_galfit.csv', 'allfields/uds/uds_3dhst.v4.1_f160w_galfit.csv']
mass_files = ['3D-HST_catalogs/aegis/Grism/aegis_3dhst.v4.1.5.zbest.fout', '3D-HST_catalogs/cosmos/Grism/cosmos_3dhst.v4.1.5.zbest.fout', '3D-HST_catalogs/goodsn/Grism/goodsn_3dhst.v4.1.5.zbest.fout', '3D-HST_catalogs/goodss/Grism/goodss_3dhst.v4.1.5.zbest.fout', '3D-HST_catalogs/uds/Grism/uds_3dhst.v4.1.5.zbest.fout']

prospector = pd.read_csv('prospector_3dhst_catalog_with_UVJ_galfit.csv')

df_list = []

for i, filename in enumerate(input_files):
    fields = pd.read_csv (filename, sep ='\s+')
    fields = fields.astype({"NUMBER": str})
    fields["NUMBER"] = filename.split("/")[1].upper()+ '_' + fields["NUMBER"]
    fields.rename(columns={'NUMBER': 'objname'}, inplace=True)
    if filename.endswith("f125w.galfit"):
        fields.rename(columns={'re': 're_f125w', 'dre':'dre_f125w', 'f':'f_f125w', 'mag': 'mag_f125w', 'dmag':'dmag_f125w'}, inplace=True)
        fields = fields[['objname','f_f125w','re_f125w','dre_f125w','mag_f125w','dmag_f125w']]
    else:
        fields.rename(columns={'mag': 'mag_f160w', 'dmag':'dmag_f160w'}, inplace=True)
        fields = fields[['objname','f','mag_f160w','dmag_f160w']]
    prospector_index = prospector.set_index('objname')
    fields_index = fields.set_index('objname')
    prospector = prospector_index.reindex(columns=prospector_index.columns.union(fields_index.columns))
    prospector.update(fields_index)
    prospector.reset_index(inplace=True)
    #print(prospector.head(1))
    
for i, filename in enumerate(mass_files):
    masses = pd.read_csv (filename, sep ='\s+')
    masses = masses.astype({"id": str})
    masses["id"] = filename.split("/")[1].upper()+ '_' + masses["id"]
    masses.rename(columns={'id': 'objname', 'lmass': 'FAST_lmass'}, inplace=True)
    #if filename.endswith("f125w.galfit"):
        #masses.rename(columns={'re': 're_f125w', 'dre':'dre_f125w', 'f':'f_f125w'}, inplace=True)
    masses = masses[['objname','FAST_lmass']]
    #else:
        #masses = masses[['objname','f']]
    prospector_index = prospector.set_index('objname')
    masses_index = masses.set_index('objname')
    prospector = prospector_index.reindex(columns=prospector_index.columns.union(masses_index.columns))
    prospector.update(masses_index)
    prospector.reset_index(inplace=True)
    #print(prospector.head(1))

cols = ['objname', 'f', 'f_f125w', 'z', 'UV', 'VJ', 're', 'dre', 're_f125w', 'dre_f125w', 'mag_f125w','dmag_f125w', 'mag_f160w','dmag_f160w', 'FAST_lmass', 'logstellar_mass_median', 'logstellar_mass_errup', 'logstellar_mass_errdown', 'sfr_100myr_median', 'sfr_100myr_errup', 'sfr_100myr_errdown', 'logssfr_100myr_median', 'logssfr_100myr_errup', 'logssfr_100myr_errdown', 'logzsol_median', 'logzsol_errup', 'logzsol_errdown', 'tau_dust_median', 'tau_dust_errup', 'tau_dust_errdown', 'avg_age_median', 'avg_age_errup', 'avg_age_errdown', 'q', 'dq']
prospector = prospector[cols]
print(prospector.head(1))
prospector.to_csv('prospector_UVJ_galfit_FAST.csv')

    objname    f  f_f125w       z        UV       VJ        re       dre  \
0  AEGIS_13  0.0      1.0  1.3049  1.651843  1.90384  0.388926  0.006917   

   re_f125w  dre_f125w  ...  logzsol_errup  logzsol_errdown  tau_dust_median  \
0   1.55208   0.092354  ...        -0.0107          -0.3838           1.5256   

   tau_dust_errup  tau_dust_errdown  avg_age_median  avg_age_errup  \
0          1.6387            1.3251          2.3332         2.6073   

   avg_age_errdown      q        dq  
0           2.0686  0.486  0.012781  

[1 rows x 35 columns]


In [ ]:
for i, filename in enumerate(mass_files):
    masses = pd.read_csv (filename, sep ='\s+')
    masses = masses.astype({"id": str})
    masses["id"] = filename.split("/")[1].upper()+ '_' + masses["id"]
    masses.rename(columns={'id': 'objname', 'lmass': 'FAST_lmass'}, inplace=True)
    #if filename.endswith("f125w.galfit"):
        #masses.rename(columns={'re': 're_f125w', 'dre':'dre_f125w', 'f':'f_f125w'}, inplace=True)
    masses = masses[['objname','FAST_lmass']]
    #else:
        #masses = masses[['objname','f']]